# Arbitrary number of inputs with binary outcome

Here where are testing the linear algorithm, that we previously used for a binary input / binary output case on an
arbitrary number of inputs / binary output case. As a non-local probability distribution we will use the *generalised
PR-box*.

Furthermore we will extend the number of outputs to 3, to simulate detectors with finite efficiency.

In [40]:
from itertools import product
inputs_a = range(4)
inputs_b = range(4)
outputs = range(2)
dim = (len(outputs)**2) * (len(inputs_a)*len(inputs_b))
# a list of all possible input and output combinations
output_input_combs = product(outputs, outputs, inputs_a, inputs_b)

# Generalised PR Box
We are now taking the generalised PR box distribution and find a Bell inequality that it violates.

In [41]:
from utils import general_pr_box, get_deterministic_behaviors
import numpy as np
from scipy.optimize import linprog

# define the behavior corresponding to the Bell inequality
p = []
for c in output_input_combs:
    p.append(general_pr_box(*c))
p = np.array(p)
assert p.shape[0] == dim

# get the deterministic behaviors for this case
dets = get_deterministic_behaviors(inputs_a, inputs_b, outputs)

# setup the vectors for SciPy solver
p = np.r_[p, [-1.0]]
dets = np.c_[dets, -1.0 * np.ones(dets.shape[0])]
assert dets.shape[1] == dim+1

# objective function and inequalities
obj = -p
lhs_ineq = np.append(dets, [p], axis=0)
rhs_ineq = np.r_[np.zeros(dets.shape[0]), [1.0]]
print()
# run the optimizer
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq)
print(opt.message)

print('S = s*p - S_L = {}'.format(p @ opt.x))


Optimization terminated successfully.
S = s*p - S_L = 1.0000000017794832


We find that the PR-box behavior fulfills  $S > 0$. Thus it is non-local!

## Extension to a inefficient detector
We now reserve the output $a = 2$ or $b = 2$ to be a failure of the detector.
Thus we have to redefine the probability distribution and the deterministic behaviors.


In [42]:
from utils import general_pr_box_extended
# inputs and outputs
inputs_a = range(4)
inputs_b = range(4)
outputs = range(3) # 2 outputs + 1 failure
outputs_wo_failure = outputs[:-1]
dim = (len(outputs)**2) * (len(inputs_a)*len(inputs_b))
# a list of all possible input and output combinations
output_input_combs = product(outputs, outputs, inputs_a, inputs_b)

# detection efficiency
eta = 0.4

# define the probability distribution
p = []
for c in output_input_combs:
    p.append(general_pr_box_extended(*c,eta,inputs_a,inputs_b,outputs_wo_failure))
p = np.array(p)
assert p.shape[0] == dim

We now defined the PR box for a given number of inputs $m_a$ and $m_b$ and detection efficiency $\eta$. For some specific
cases of $m_a,m_b$ the lowest efficiency $\eta^{*}$, until which non local distributions exist has been found. We have set
efficiency below the threshold for the $m_a = m_b = 4$ case (threshold is $0.5$ for this case). Thus our PR-box example should be local. This is what we test
in the next code block.

In [43]:
# get local deterministic behaviors
dets = get_deterministic_behaviors(inputs_a, inputs_b, outputs)

# setup the vectors for SciPy solver
p = np.r_[p, [-1.0]]
dets = np.c_[dets, -1.0 * np.ones(dets.shape[0])]
assert dets.shape[1] == dim+1

# objective function and inequalities
obj = -p
lhs_ineq = np.r_[dets, [p]]
rhs_ineq = np.r_[np.zeros(dets.shape[0]), [1.0]]

# run the optimizer
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq)
print(opt.message)

print('S = s*p - S_L = {}'.format(p @ opt.x))

Optimization terminated successfully.
S = s*p - S_L = 3.125960462702082e-10


So as we have set $\eta \leq \eta^{*}$, we expected, that the behavior is local. Thus no Bell inequality that is violated
can be found, i.e. $S = 0$.
